## Introduction

I present a reimplementation of the following paper:

> Seppecher, P. (2012). Flexibility of wages and macroeconomic instability in an agent-based computational model with endogenous money. Macroeconomic Dynamics, 16(S2), 284-297.

After replicating the results of the original paper, I present some new questions to examine which test the robustness of the original model.

## Original model

The original paper implements the simulation using Jamel v1 software written by the author. Here I present a top-down reimplementation of the model from the original paper, with the following adjustments:

* Cash values are represented as cents and stored as `int` type, to prevent rounding and representation issues with floating-point numbers.
* Real-valued parameters are likewise represented as an arbitrary-precision `Fraction` type.
* All entities are immutable [why?]
* Entity types have no methods. Instead entity types are purely data stores, and all model mechanics are represented as separate functions. This allows me to write the code in a less fragmented manner, improving readability.

In [195]:
import enum
from enum import Enum
from fractions import Fraction
from functools import cmp_to_key, reduce
from math import floor
from random import random, randrange, sample, shuffle
from statistics import mean
from typing import NamedTuple

In [182]:
class LoanQuality(Enum):
    GOOD = enum.auto()

class Loan(NamedTuple):
    id: int
    quality: LoanQuality
    initial_value: int
    principal: int
    time_remaining: int
    interest_rate: Fraction

class Account(NamedTuple):
    id: int
    balance: int = 0
    loans: dict[int, Loan] = {}
    loans_issued: int = 0

class Bank(NamedTuple):
    capital: int = 0
    accounts: dict[int, Account] = {}
    target_capital_ratio: Fraction = Fraction(10, 100)
    propensity_to_distribute_excess_capital: Fraction = Fraction(50, 100)
    loan_normal_interest_rate: Fraction = Fraction(5, 100)
    loan_term: int = 12
    loan_normal_rate: Fraction = Fraction(5, 100)
    loan_penalty_rate: Fraction = Fraction(10, 100)

class EmploymentStatus(Enum):
    EMPLOYED = enum.auto()
    INVOLUNTARY_UNEMPLOYED = enum.auto()
    VOLUNTARY_UNEMPLOYED = enum.auto()

class Household(NamedTuple):
    id: int
    account_id: int = None
    firm_id: int = None
    employment_contract_id: int = None
    employment_status: EmploymentStatus = EmploymentStatus.INVOLUNTARY_UNEMPLOYED
    unemployment_duration: int = 0
    consumption_quantity: int = 0
    consumption_value: int = 0
    savings_target_ratio: Fraction = Fraction(5, 100)
    propensity_to_save: Fraction = Fraction(5, 100)
    propensity_to_consume_excess: Fraction = Fraction(50, 100)
    wage_resistance: int = 12
    wage_flexibility: Fraction = Fraction(5, 100)
    job_search_size: int = 5
    purchase_search_size: int = 5

class EmploymentContract(NamedTuple):
    id: int
    household_id: int
    firm_id: int
    time_remaining: int
    wage: int

class Machine(NamedTuple): # Integrated
    id: int
    progress: int = 0
    value: int = 0
    production_time: int = 6
    productivity: int = 100

class Firm(NamedTuple):
    id: int
    bankrupt: bool = False
    employment_contracts: dict[int, EmploymentContract] = {}
    employment_contracts_issued: int = 0
    hiring_vacancies = 10
    hiring_wage: int = 3_000_00
    hiring_wage_flex_up: Fraction = Fraction(6, 100)
    hiring_wage_flex_down: Fraction = Fraction(9, 100)
    hiring_wage_minimum: int = 0
    hiring_contract_length_minimum: int = 6
    hiring_contract_length_maximum: int = 18
    hiring_vacancy_rate_history = []
    hiring_vacancies_normal_rate: Fraction = Fraction(3, 100)
    machinery: dict[int, Machine] = {i : Machine(id=i) for i in range(10)}
    inventory_volume: int = 0
    inventory_value: int = 0
    inventory_for_sale_quantity: int = 0
    inventory_for_sale_unit_price: int = None
    target_capital_ratio: Fraction = Fraction(10, 100)
    propensity_to_distribute_excess_capital: Fraction = Fraction(50, 100)
    target_inventory: int = 4_000
    propensity_to_sell: Fraction = Fraction(50, 100)
    machines: int = 10

class Simulation(NamedTuple):
    bank: Bank = Bank()
    households: dict[int, Household] = {i : Household(id=i) for i in range(1_000)}
    firms: dict[int, Firm] = {i : Firm(id=i) for i in range(100)}

In [1]:
def step(sim):
    sim = bank_pay_dividend(sim)
    sim = firms_pay_dividend(sim)
    sim = firms_plan_production(sim)
    sim = households_job_search(sim)
    sim = firms_production(sim)
    sim = households_consume(sim)
    sim = bank_debt_recovery(sim)
    return sim

In [122]:
def account_debt(account):
    return sum([loan.principal for loan in account.loans])

def bank_pay_dividend(sim):
    bank = sim.bank
    households = sim.households.values()
    dividend = bank_dividend(bank=bank)
    household = sample(households, 1)[0]
    # TODO this is not how dividends are supposed to work!
    household_account = bank.accounts[household.account_id]
    return sim._replace(
        bank = bank._replace(
           capital = bank.capital - dividend,
           accounts = {
               **bank.accounts,
               **{
                   household_account.id : household_account._replace(
                       balance = household_account.balance + dividend
                   )
               }
           }
        )
    )

def bank_dividend(bank):
    required_capital = bank.target_capital_ratio * bank_total_assets(bank=bank)
    excedent_capital = max(0, bank.capital - required_capital)
    return floor(excedent_capital * bank.propensity_to_distribute_excess_capital)

def bank_total_assets(bank):
    return sum([account_debt(account) for account in bank.accounts.values()])

In [124]:
def firms_pay_dividend(sim):
    firm_ids = [id for (id, firm) in sim.firms.items() if not firm.bankrupt]
    shuffle(firm_ids)
    reduce(firm_pay_dividend, firm_ids, sim)

def firm_pay_dividend(sim, firm):
    firm_account = sim.bank.accounts[firm.account_id]
    dividend = firm_dividend(firm=firm, firm_account=firm_account)
    household = sample(sim.households, 1)[0]
    household_account = sim.bank.accounts[household.account_id]
    return sim._replace(
        bank = bank._replace(
           accounts = {
               **bank.accounts,
               **{
                   household_account.id : household_account._replace(
                       balance = household_account.balance + dividend
                   ),
                   firm_account.id : firm_account._replace(
                       balance = firm_account.balance - dividend
                   )
               }
           }
        )
    )

def firm_dividend(firm, firm_account):
    assets = firm_account.balance + firm.inventory_value
    capital = assets - account_debt(firm_account)
    capital_target = firm.target_capital_ratio * assets
    excedent_capital = max(0, capital - capital_target)
    dividend = floor(excedent_capital * firm.propensity_to_distribute_excess_capital)
    return min(firm_account.balance, dividend)

In [ ]:
def bank_lend(bank, account_id, principal):
    account = bank.accounts[account_id]
    return bank._replace(
        accounts = {
            **bank.accounts,
            account_id: account._replace(
                loans_issued = account.loans_issued + 1
                loans = [
                    *account.loans,
                    Loan(
                        id = account.loans_issued,
                        principal = principal,
                        initial_value = principal,
                        quality = LoanQuality.GOOD,
                        interest_rate = bank.loan_normal_interest_rate,
                        time_remaining = bank.loan_term
                    )
                ],
                balance = account.balance + principal
            )
        }
    )

def firms_plan_production(sim):
    firm_ids = [id for (id, firm) in sim.firms.items() if not firm.bankrupt]
    shuffle(firm_ids)
    reduce(firm_plan_production, sim.firms.keys(), sim)

def firm_plan_production(sim, firm_id):
    bank = sim.bank
    firm = sim.firms[firm_id]
    
    inventory_ratio = Fraction(firm.inventory_volume, firm.target_inventory)
        
    # Determine production level
    max_production = sum([machine.productivity for machine in firm.machinery])
    alpha1 = random()
    alpha2 = random()
    utilization_ratio_delta = alpha1 * firm.utilization_ratio_flexibility
    target_utilization_ratio = firm.target_utilization_ratio
    if inventory_ratio < 1 - alpha1 * alpha2:
        # Low level
        target_utilization_ratio += utilization_ratio_delta
    elif 1 + alpha1 * alpha2 < inventory_ratio:
        # High level
        target_utilization_ratio -= utilization_ratio_delta
    # TODO? rectified
    target_utilization_ratio = min(1, target_utilization_ratio, firm.max_utilization_ratio)
    
    # Determine price
    inventory_for_sale_unit_price = firm.inventory_for_sale_unit_price
    if inventory_for_sale_unit_price == 0:
        unit_cost = Fraction(inventory_value, inventory_volume)
        inventory_for_sale_unit_price = 1 + 0.5 * random() * unit_cost
    else:
        alpha1 = random()
        alpha2 = random()
        if inventory_ratio < 1 - alpha1 * alpha2:
            # Low level
            inventory_for_sale_unit_price *= 1 + alpha1 * firm.price_flexibility
        elif 1 + alpha1 * alpha2 < inventory_ratio:
            # High level
            inventory_for_sale_unit_price *= 1 - alpha1 * firm.price_flexibility
        inventory_for_sale_unit_price = max(1, price)
    
    # Determine workforce
    target_employees = len(firm.machinery) * target_utilization_ratio
    current_employees = len([employment_contract
                             for employment_contract
                             in firm.employment_contracts.values()
                             if 0 < employment_contract.time_remaining])
    target_hiring = target_employees - current_employees
    if target_hiring < 0:
        # TODO layoff
    
    # Determine wage
    hiring_wage = firm.hiring_wage
    if 0 < target_hiring:
        alpha1 = random()
        alpha2 = random()
        #average_vacancies = mean(firm.hiring_vacancies_history)
        average_vacancy_rate = mean(firm.hiring_vacancy_rate_history)
        vacancy_ratio = Fraction(average_vacancy_rate, firm.hiring_vacancies_normal_rate)
        if vacancy_ratio < 1 - alpha1 * alpha2:
            # Low level
            hiring_wage *= 1 + alpha1 * firm.hiring_wage_flex_down
        elif 1 + alpha1 * alpha2 < vacancy_ratio:
            # High level
            hiring_wage *= 1 - alpha1 * firm.hiring_wage_flex_up
        hiring_wage = max(hiring_wage, firm.hiring_wage_minimum)
    
    wage_budget = target_hiring * hiring_wage + sum([employment_contract.wage
                                                     for employment_contract
                                                     in firm.employment_contracts.values()
                                                     if 0 < employment_contract.time_remaining])
    
    # Determine finance
    production_budget = wage_budget # No materials required, IntegratedFactory
    financing_need = production_budget - bank.accounts[firm.account_id].balance
    if 0 < financing_need:
        bank = bank_lend(bank=bank, account_id=firm.account_id, principal=financing_need)

    return sim._replace(
        bank = bank,
        firms = {
            **sim.firms,
            firm_id : firm._replace(
                target_utilization_ratio = target_utilization_ratio,
                inventory_for_sale_unit_price = inventory_for_sale_unit_price,
                target_hiring = target_hiring,
                hiring_wage = hiring_wage
                # TODO update hiring_vacancy_rate_history
            )
        }
    )

In [ ]:
def households_job_search(sim):
    household_ids = list(sim.households.keys())
    shuffle(household_ids)
    reduce(household_job_search, household_ids, sim)

def household_job_search(sim, household_id):
    household = sim.households[household_id]
    employment_contract = sim.firms[household.firm_id].employment_contracts[household.employment_contract_id]
    
    if employment_contract.time_remaining == 0:
        employment_contract = None
    
    reservation_wage = household.reservation_wage
    unemployment_duration = household.unemployment_duration
    if employment_contract:
        reservation_wage = employment_contract.wage
        unemployment_duration = 0
    else:
        unemployment_duration += 1
        alpha1 = random()
        alpha2 = random()
        if alpha1 * household.wage_resistance < unemployment_duration:
            reservation_wage *= 1 - household.wage_flexibility * alpha2
    
    employment_status = household.employment_status
    if employment_contract:
        employment_status = EmploymentStatus.EMPLOYED
    else:
        employment_status = EmploymentStatus.INVOLUNTARY_UNEMPLOYED
        employers = household_search_employers(sim.firms)
        best_employer = employers[0]
        if reservation_wage <= best_employer.hiring_wage:
            sim = firm_hire_household(sim, best_employer.id, household_id)
            employment_contract = sim.firms[household.firm_id].employment_contracts[household.employment_contract_id]
            employment_status = EmploymentStatus.EMPLOYED
        else:
            employment_status = EmploymentStatus.VOLUNTARY_UNEMPLOYED
    
    return sim._replace(
        households = {
            **sim.households,
            household_id : household._replace(
                employment_contract_id = employment_contract.id,
                reservation_wage = reservation_wage,
                unemployment_duration = unemployment_duration,
                employment_status = employment_status
            )
        }
    )
    
def household_search_employers(household, firms):
    hiring_firms = list(filter(
        lambda firm: not firm.bankrupt and 0 < firm.hiring_vacancies,
        firms.values()
    ))
    found_firms = sample(hiring_firms, household.job_search_size)
    return sorted(found_firms, key=lambda firm: firm.hiring_wage, reverse=true)

def firm_hire_household(sim, firm_id, household_id):
    firm = sim.firms[firm_id]
    household = sim.households[household_id]
    employment_contract = EmploymentContract(
        id = firm.employment_contracts_issued,
        household_id = household_id
        firm_id = firm_id
        time_remaining = randrange(firm.hiring_contract_length_minimum, firm.hiring_contract_length_minimum + 1)
        wage = firm.hiring_wage
    )
    return sim._replace(
        firms = {
            **sim.firms,
            firm_id : firm._replace(
                employment_contracts = {
                    **firm.employment_contracts,
                    employment_contract.id : employment_contract
                },
                employment_contracts_issued = firm.employment_contracts_issued + 1,
                hiring_vacancies = firm.hiring_vacancies - 1
            )
        },
        households = {
            **sim.households,
            household_id : household._replace(
                firm_id = firm_id
                employment_contract_id = employment_contract.id,
                employment_status = EmploymentStatus.EMPLOYED
            )
        }
    )

In [ ]:
def firms_production(sim):
    firm_ids = list(sim.firms.keys())
    shuffle(firm_ids)
    reduce(firm_production, firm_ids, sim)

def firm_production(sim, firm_id):
    firm = sim.firms[firm_id]
    
    # Pay workers
    # TODO count vacancies (again?) ?
    sim = reduce(lambda sim, employment_contract_id: firm_pay_worker(sim, firm_id, employment_contract_id),
                 firm.employment_contracts.keys(),
                 sim)
    
    # Produce
    def machine_sort_cmp(machine_x, machine_y):
        if machine_x.progress > machine_y.progress:
            return -1
        elif machine_x.progress < machine_y.progress:
            return 1
        elif machine_x.productivity > machine_y.productivity:
            return -1
        elif machine_x.productivity < machine_y.productivity:
            return 1
        else:
            return 0
    machinery = sorted(firm.machinery.values(), key=cmp_to_key(machine_sort_cmp))
    employees = [employment_contract
                 for employment_contract
                 in firm.employment_contracts
                 if 0 < employment_contract.time_remaining]
    work_schedule = zip(machinery, employees)
    def machine_work(firm, machine_employee):
        machine, employment_contract = machine_employee
        progress = machine.progress + 1
        value = machine.value + employment_contract.wage
        inventory_value = firm.inventory_value
        inventory_volume = firm.inventory_volume
        if progress == machine.production_time:
            inventory_value += value
            inventory_volume += machine.productivity
            progress = 0
            value = 0
        return firm._replace(
            machinery = {
                **firm.machinery,
                machine.id : machine._replace(
                    progress = progress,
                    value = value,
                )
            },
            inventory_value = inventory_value,
            inventory_volume = inventory_volume
        )
    firm = reduce(machine_work, work_schedule, firm)
    
    # Offer goods
    max_production = sum([machine.productivity for machine in firm.machinery])
    inventory_for_sale_volume = min(firm.inventory_volume * firm.propensity_to_sell,
                                    2 * max_production)
    
    return sim._replace(
        firms = sim.firms._replace(
            firm.id : firm._replace(
                inventory_for_sale_volume = inventory_for_sale_volume
            )
        )
    )

def firm_pay_worker(sim, firm_id, employment_contract_id):
    firm = sim.firms[firm_id]
    firm_account = sim.bank.accounts[firm.account_id]
    employment_contract = firm.employment_contracts[employment_contract_id]
    household = sim.households[employment_contract.household_id]
    household_account = sim.bank.accounts[household.account_id]
    # TODO update wage bill and effective workforce?
    return sim._replace(
        bank = sim.bank._replace(
            accounts = {
                **sim.bank.accounts,
                firm_account.id : firm_account._replace(
                    balance = firm_account.balance - employment_contract.wage
                ),
                household_account.id : household_account._replace(
                    balance = household_account.balance + employment_contract.wage
                )
            }
        )
    )

In [ ]:
def households_consume(sim):
    household_ids = list(sim.households.keys())
    shuffle(firm_ids)
    reduce(firm_production, firm_ids, sim)

def household_consume(sim, household_id):
    household = sim.households[household_id]
    account = sim.bank.accounts[household.account_id]
    
    average_income = mean(household.income_history)
    savings_target = 12 * average_income * household.savings_target_ratio
    savings = account.balance - average_income
    consumption_target = 0
    if savings < savings_target:
        consumption_target = average_income * (1 - household.propensity_to_save)
    else:
        consumption_target = average_income +
                             (savings - savings_target) * household.propensity_to_consume_excess_savings
    budget = min(account.balance, consumption_target)
    suppliers = household_search_suppliers(sim.firms)
    def purchase(sim, household_id, firm):
        household = sim.households[household_id]
        household_account = sim.bank.accounts[household.account_id]
        firm_account = sim.bank.accounts[firm.account_id]
        return sim._replace(
            bank = sim.bank._replace(
                accounts = {
                    **sim.bank.accounts,
                    household_account.id : household_account._replace(
                        balance = 
                    ),
                    firm_account.id : firm_account._replace(
                        balance = 
                    )
                }
            ),
            firms = {
                **sim.firms,
                firm.id : firm._replace(
                    inventory_volume = 
                    inventory_value = 
                    inventory_for_sale_quantity = 
                )
            }
        )
    sim = reduce(lambda sim, firm: purchase(sim, household_id, firm), suppliers, sim)

def household_search_suppliers(firms):
    selling_firms = list(filter(
        lambda firm: 0 < firm.inventory_for_sale_price,
        firms.values()
    ))
    found_firms = sample(selling_firms, household.consumption_search_size)
    return sorted(found_firms, key=lambda firm: firm.inventory_for_sale_price, reverse=true)

In [ ]:
def bank_debt_recovery(sim):
    # TODO if bankrupt raise Exception("Bank is bankrupt.")

In [229]:
def foo(x, foo):
    y, z = foo
    return [x, y, z]
foo(1, (2, 3))

[1, 2, 3]

In [119]:
sim = Simulation(
    bank = Bank(
        capital = 1_000_00,
        accounts = {
            1: Account(
                id = 1,
                balance = 0,
                loans = [
                    Loan(
                        principal = 10
                    )
                ]
            )
        }
    ),
    households = [
        Household(
            account_id = 1
        )
    ]
)        

sim._replace(**bank_pay_dividend(**sim._asdict()))

Simulation(bank=Bank(capital=100000, accounts={1: Account(id=1, balance=49999, loans=[Loan(principal=10)])}, target_capital_ratio=Fraction(1, 10), propensity_to_distribute_excess_capital=Fraction(1, 2)), households=[Household(account_id=1)])